# Proyecto Final Visión

## Setup

In [77]:
#Importar las librerías por utilizar
import os
import numpy as np
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import time
from skimage import io
from playsound import playsound
from skimage.filters import threshold_otsu
from skimage.transform import resize, probabilistic_hough_line,hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from skimage.color import rgb2gray,rgb2hsv,hsv2rgb
from skimage.draw import circle_perimeter
from skimage import exposure
from skimage.morphology import disk
from skimage.morphology import (erosion, dilation, opening, closing)
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table
from skimage import data, img_as_float
from skimage import exposure
from scipy import ndimage
import pandas as pd

In [2]:
#Obtener la direccion de las imagenes
path = os.getcwd()

#Lectura de las imagenes de prueba
folder = 'MUESTRAS'
folder_path = os.path.join(path, folder)
files = os.listdir(folder_path)
lista_fotos = []
for file in files:
    file_path = os.path.join(folder_path, file)
    foto = io.imread(file_path)
    lista_fotos.append(foto)

In [3]:
#PARAMETROS

#Ejecutar para informe
INF = False

#Recortar la imagen(%)
Y_MIN = 0.5
Y_MAX = 1
X_MIN = 0
X_MAX = 1

#Escalado
X_PX = 320
Y_PX = 360

#numero de colores incluyendo el fondo
N_COLORS = 5

## Preprocesado

In [4]:
# Escala y Recorta la imagen a 360x320
def edit_image(image):
    y,x = image.shape[:2]
    cropped = image[int(y*Y_MIN):int(y*Y_MAX),int(x*X_MIN):int(x*X_MAX)]
    resized = resize(cropped,(Y_PX,X_PX),preserve_range=True).astype(int)
    return resized

In [5]:
# Convierte la imagen RGBA a RGB
def rgba2rgb( rgba, background=(255,255,255) ):
    row, col, ch = rgba.shape
    if ch == 3:
        return rgba
    assert ch == 4, 'RGBA image has 4 channels.'
    rgb = np.zeros( (row, col, 3), dtype='float32' )
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]
    a = np.asarray( a, dtype='float32' ) / 255.0
    R, G, B = background
    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B
    return np.asarray( rgb, dtype='uint8' )

In [6]:
# Evaluación y Mejora de Saturacion
# https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html
matplotlib.rcParams['font.size'] = 8

def plot_img_and_hist(image, axes, bins=256):
    #Plot an image along with its histogram and cumulative histogram.
    image = img_as_float(image)
    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display image
    ax_img.imshow(image, cmap=plt.cm.gray)
    ax_img.set_axis_off()

    # Display histogram
    ax_hist.hist(image.ravel(), bins=bins, histtype='step', color='black')
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')
    ax_hist.set_xlim(0, 1)
    ax_hist.set_yticks([])

    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(image, bins)
    ax_cdf.plot(bins, img_cdf, 'r')
    ax_cdf.set_yticks([])

    return ax_img, ax_hist, ax_cdf

def stretch_hsv(img_hsv,channel):
    p2, p98 = np.percentile(img_hsv[:,:,channel], (2, 98))
    stretched = exposure.rescale_intensity(img_hsv[:,:,channel], in_range=(p2, p98))
    return stretched

if INF:
    img_org_HSV = rgb2hsv(rgba2rgb(edit_image(lista_fotos[np.random.randint(0,len(lista_fotos))])))
    fig_eval, axs = plt.subplots(1,3,figsize=(22,5))
    fig_eval.suptitle('Evaluación de imagen de entrada')

    axs[0].set_title("Imagen de entrada")
    axs[0].imshow(img_org_HSV[:,:,1],cmap=plt.cm.gray)
    axs[0].set_axis_off()

    # create the histogram
    histogram, bin_edges = np.histogram(img_org_HSV[:,:,1], bins=256, range=(0, 1))
    # Display histogram
    axs[1].set_title("Histograma de Saturación")
    axs[1].set_xlabel("Valor de Saturación")
    axs[1].set_ylabel("Número de pixeles")
    axs[1].set_xlim([0.0, 1])
    axs[1].plot(bin_edges[0:-1], histogram)
    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(img_org_HSV[:,:,1], 256)
    axs[2].set_title("Distribución acumulativa")
    axs[2].set_xlabel("Valor de Saturación")
    axs[2].set_ylabel("Acumulado")
    axs[2].set_xlim([0.0, 1])
    axs[2].plot(bins, img_cdf, 'r')
    axs[2].set_yticks([])

    plt.show()

    # Load an example image
    img_sat = img_org_HSV[:,:,1]

    # Contrast stretching
    p2, p98 = np.percentile(img_sat, (2, 98))
    img_rescale_sat = exposure.rescale_intensity(img_sat, in_range=(p2, p98))

    # Equalization
    img_eq_sat = exposure.equalize_hist(img_sat)

    # Adaptive Equalization
    img_adapteq_sat = exposure.equalize_adapthist(img_sat, clip_limit=0.03)

    # Display results
    fig = plt.figure(figsize=(22, 10))
    axes = np.zeros((2, 4), dtype=np.object)
    axes[0, 0] = fig.add_subplot(2, 4, 1)
    for i in range(1, 4):
        axes[0, i] = fig.add_subplot(2, 4, 1+i, sharex=axes[0,0], sharey=axes[0,0])
    for i in range(0, 4):
        axes[1, i] = fig.add_subplot(2, 4, 5+i)

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_sat, axes[:, 0])
    ax_img.set_title('Low contrast image')

    y_min, y_max = ax_hist.get_ylim()
    ax_hist.set_ylabel('Number of pixels')
    ax_hist.set_yticks(np.linspace(0, y_max, 5))

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_rescale_sat, axes[:, 1])
    ax_img.set_title('Stretching de saturación')

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_eq_sat, axes[:, 2])
    ax_img.set_title('Ecualización de Saturación')

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_adapteq_sat, axes[:, 3])
    ax_img.set_title('Ecualización adaptativa')

    ax_cdf.set_ylabel('Acumulado')
    ax_cdf.set_yticks(np.linspace(0, 1, 5))

    # prevent overlap of y-axis labels
    fig.tight_layout()
    plt.show()

In [7]:
# Evaluación y Mejora de Contraste
# https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html
if INF:
    img_org_HSV = rgb2hsv(rgba2rgb(edit_image(lista_fotos[np.random.randint(0,len(lista_fotos))])))
    fig_eval, axs = plt.subplots(1,3,figsize=(22,5))
    fig_eval.suptitle('Evaluación de imagen de entrada')

    axs[0].set_title("Imagen de entrada")
    axs[0].imshow(img_org_HSV[:,:,1],cmap=plt.cm.gray)
    axs[0].set_axis_off()

    # create the histogram
    histogram, bin_edges = np.histogram(img_org_HSV[:,:,2], bins=256, range=(0, 1))
    # Display histogram
    axs[1].set_title("Histograma de Escala de grises")
    axs[1].set_xlabel("Valor de Gradiente")
    axs[1].set_ylabel("Número de pixeles")
    axs[1].set_xlim([0.0, 1])
    axs[1].plot(bin_edges[0:-1], histogram)
    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(img_org_HSV[:,:,2], 256)
    axs[2].set_title("Distribución acumulativa")
    axs[2].set_xlabel("Valor de Gradiente")
    axs[2].set_ylabel("Acumulado")
    axs[2].set_xlim([0.0, 1])
    axs[2].plot(bins, img_cdf, 'r')
    axs[2].set_yticks([])

    plt.show()

    # Load an example image
    img_con = img_org_HSV[:,:,2]

    # Contrast stretching
    p2, p98 = np.percentile(img_con, (2, 98))
    img_rescale_con = exposure.rescale_intensity(img_con, in_range=(p2, p98))

    # Equalization
    img_eq_con = exposure.equalize_hist(img_con)

    # Adaptive Equalization
    img_adapteq_con = exposure.equalize_adapthist(img_con, clip_limit=0.03)

    # Display results
    fig = plt.figure(figsize=(22, 10))
    axes = np.zeros((2, 4), dtype=np.object)
    axes[0, 0] = fig.add_subplot(2, 4, 1)
    for i in range(1, 4):
        axes[0, i] = fig.add_subplot(2, 4, 1+i, sharex=axes[0,0], sharey=axes[0,0])
    for i in range(0, 4):
        axes[1, i] = fig.add_subplot(2, 4, 5+i)

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_con, axes[:, 0])
    ax_img.set_title('Low contrast image')

    y_min, y_max = ax_hist.get_ylim()
    ax_hist.set_ylabel('Number of pixels')
    ax_hist.set_yticks(np.linspace(0, y_max, 5))

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_rescale_con, axes[:, 1])
    ax_img.set_title('Stretching de contraste')

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_eq_con, axes[:, 2])
    ax_img.set_title('Ecualización de Contraste')

    ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_adapteq_con, axes[:, 3])
    ax_img.set_title('Ecualización adaptativa')

    ax_cdf.set_ylabel('Acumulado')
    ax_cdf.set_yticks(np.linspace(0, 1, 5))

    # prevent overlap of y-axis labels
    fig.tight_layout()
    plt.show()

In [8]:
# Resultados de Imagen mejorada
if INF:
    index = np.random.randint(0,len(lista_fotos))
    print('IMG_' + str(index+1) + '.png')
    img_org_RGB = rgba2rgb(edit_image(lista_fotos[index]))
    img_org_HSV = rgb2hsv(img_org_RGB)
    img_eq_1 = img_org_HSV
    img_eq_1[:,:,1] = stretch_hsv(img_org_HSV,1)
    img_org_HSV = rgb2hsv(img_org_RGB)
    
    img_eq_2 = img_org_HSV
    img_eq_2[:,:,2] = stretch_hsv(img_org_HSV,2)
    img_org_HSV = rgb2hsv(img_org_RGB)

    img_eq_3 = img_org_HSV
    img_eq_3[:,:,1] = stretch_hsv(img_org_HSV,1)
    img_eq_3[:,:,2] = stretch_hsv(img_org_HSV,2)
    img_org_HSV = rgb2hsv(img_org_RGB)

    fig1, axs = plt.subplots(1,4,figsize=(17,6))
    fig1.suptitle('Filtrado de ruido, suavizado y Recuperación de bordes')

    axs[0].set_title("Imagen original")
    axs[0].imshow(hsv2rgb(img_org_HSV))
    axs[0].set_axis_off()

    axs[1].set_title("Imagen mejora saturacion")
    axs[1].imshow(hsv2rgb(img_eq_1))
    axs[1].set_axis_off()

    axs[2].set_title("Imagen mejora contraste")
    axs[2].imshow(hsv2rgb(img_eq_2))
    axs[2].set_axis_off()

    axs[3].set_title("Imagen mejora saturacion y contraste")
    axs[3].imshow(hsv2rgb(img_eq_3))
    axs[3].set_axis_off()

    plt.tight_layout()
    plt.show()

In [9]:
# Quantificacion 4 colores
#https://scikit-learn.org/stable/auto_examples/cluster/plot_color_quantization.html#sphx-glr-auto-examples-cluster-plot-color-quantization-py

def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    return codebook[labels].reshape(w, h, -1)

def img_to_5_colors(img):
    # Copia de la imagen por cuantificar
    img_5_colors = img

    # Convert to floats instead of the default 8 bits integer coding. Dividing by
    # 255 is important so that plt.imshow behaves works well on float data (need to
    # be in the range [0-1])
    img_5_colors = np.array(img_5_colors, dtype=np.float64)

    # Load Image and transform to a 2D numpy array.
    w, h, d = tuple(img_5_colors.shape)
    assert d == 3
    image_array = np.reshape(img_5_colors, (w * h, d))
    image_array_sample = shuffle(image_array, random_state=0, n_samples=1_000)
    kmeans = KMeans(n_clusters=N_COLORS, random_state=0).fit(image_array_sample)
    labels = kmeans.predict(image_array)
    colores = kmeans.cluster_centers_
    #coloresh = rgb2hsv(colores)[:,0]
    #coloress = rgb2hsv(colores)[:,1]
    #coloresv = rgb2hsv(colores)[:,2]
    coloresh = colores[:,0]
    coloress = colores[:,1]
    coloresv = colores[:,2]
    #print(coloresh)
    #print(coloress)
    #print(coloresv)
    #pru = recreate_image(colores, labels, w, h)
    #plt.imshow(pru)
    for i in range(N_COLORS):
        if not (0.07 < coloresh[i] < 0.19 and coloress[i] > 0.219 and coloresv[i] > 0.15):
            colores[i] = [0, 0, 0]
        new_img = recreate_image(colores, labels, w, h)
    #plt.imshow(new_img)
    new_img = rgb2hsv(new_img)[:,:,2]
    new_img = new_img > 0 #threshold_otsu(new_img)
    footprint = disk(10)
    new_img = closing(new_img,footprint)
    return new_img

In [10]:
# Resultados cuantificada, binarizada y clausura
if INF:
    index = np.random.randint(0,len(lista_fotos))
    print('IMG_' + str(index+1) + '.png')
    img_org_RGB = rgba2rgb(edit_image(lista_fotos[index]))
    img_org_HSV = rgb2hsv(img_org_RGB)
    img_eq_1 = img_org_HSV
    img_eq_1[:,:,1] = stretch_hsv(img_org_HSV,1)
    img_org_HSV = rgb2hsv(img_org_RGB)
    
    img_eq_2 = img_org_HSV
    img_eq_2[:,:,2] = stretch_hsv(img_org_HSV,2)
    img_org_HSV = rgb2hsv(img_org_RGB)

    img_eq_3 = img_org_HSV
    img_eq_3[:,:,1] = stretch_hsv(img_org_HSV,1)
    img_eq_3[:,:,2] = stretch_hsv(img_org_HSV,2)
    img_org_HSV = rgb2hsv(img_org_RGB)

    quant1 = img_to_5_colors(img_org_HSV)
    quant2 = img_to_5_colors(img_eq_1)
    quant3 = img_to_5_colors(img_eq_2)
    quant4 = img_to_5_colors(img_eq_3)

    fig1, axs = plt.subplots(1,4,figsize=(17,6))
    fig1.suptitle('Cuantificación, binarización y clausura de Imagen')

    axs[0].set_title("Imagen original")
    axs[0].imshow(quant1,cmap='gray')
    axs[0].set_axis_off()


    axs[1].set_title("Imagen mejora saturacion")
    axs[1].imshow(quant2,cmap='gray')
    axs[1].set_axis_off()

    axs[2].set_title("Imagen mejora contraste")
    axs[2].imshow(quant3,cmap='gray')
    axs[2].set_axis_off()

    axs[3].set_title("Imagen mejora saturacion y contraste")
    axs[3].imshow(quant4,cmap='gray')
    axs[3].set_axis_off()

    plt.tight_layout()
    plt.show()

In [11]:
# Función de preprocesado
def preproc(img):
    resized = edit_image(img)
    img_RGB =rgba2rgb(resized)
    img_HSV =rgb2hsv(img_RGB)
    img_eq = img_HSV
    img_eq[:,:,1] = stretch_hsv(img_HSV,1)
    img_eq[:,:,2] = stretch_hsv(img_HSV,2)
    #plt.imshow(hsv2rgb(img_eq))
    #plt.show()    
    quant = img_to_5_colors(img_HSV)
    return quant

In [12]:
# Resultado Preprocesado etapas
def img_to_5_colors_INF(img):
    # Copia de la imagen por cuantificar
    img_5_colors = img

    # Convert to floats instead of the default 8 bits integer coding. Dividing by
    # 255 is important so that plt.imshow behaves works well on float data (need to
    # be in the range [0-1])
    img_5_colors = np.array(img_5_colors, dtype=np.float64)

    # Load Image and transform to a 2D numpy array.
    w, h, d = tuple(img_5_colors.shape)
    assert d == 3
    image_array = np.reshape(img_5_colors, (w * h, d))
    image_array_sample = shuffle(image_array, random_state=0, n_samples=1_000)
    kmeans = KMeans(n_clusters=N_COLORS, random_state=0).fit(image_array_sample)
    labels = kmeans.predict(image_array)
    colores = kmeans.cluster_centers_
    #coloresh = rgb2hsv(colores)[:,0]
    #coloress = rgb2hsv(colores)[:,1]
    #coloresv = rgb2hsv(colores)[:,2]
    coloresh = colores[:,0]
    coloress = colores[:,1]
    coloresv = colores[:,2]
    #print(coloresh)
    #print(coloress)
    #print(coloresv)
    #pru = recreate_image(colores, labels, w, h)
    #plt.imshow(pru)
    for i in range(N_COLORS):
        if not (0.07 < coloresh[i] < 0.19 and coloress[i] > 0.219 and coloresv[i] > 0.15):
            colores[i] = [0, 0, 0]
        new_img = recreate_image(colores, labels, w, h)
    #plt.imshow(new_img)
    new_img1 = hsv2rgb(new_img)
    new_img = hsv2rgb(new_img)[:,:,2]
    new_img = new_img > 0#threshold_otsu(new_img)
    footprint = disk(10)
    new_img = closing(new_img,footprint)
    return new_img1, new_img

if INF:
    index = np.random.randint(0,len(lista_fotos))
    print('IMG_' + str(index+1) + '.png')
    img_org_RGB = rgba2rgb(edit_image(lista_fotos[index]))
    img_org_HSV = rgb2hsv(img_org_RGB)

    img_eq_3 = img_org_HSV
    img_eq_3[:,:,1] = stretch_hsv(img_org_HSV,1)
    img_eq_3[:,:,2] = stretch_hsv(img_org_HSV,2)
    img_org_HSV = rgb2hsv(img_org_RGB)

    quant4, quant4b = img_to_5_colors_INF(img_eq_3)

    fig1, axs = plt.subplots(1,5,figsize=(17,6))
    fig1.suptitle('Etapas principales del Preprocesado')

    axs[0].set_title("Imagen original")
    axs[0].imshow(lista_fotos[index],cmap='gray')
    axs[0].set_axis_off()


    axs[1].set_title("Imagen recortada y escalada")
    axs[1].imshow(img_org_RGB,cmap='gray')
    axs[1].set_axis_off()

    axs[2].set_title("Imagen con mejora contraste y saturación")
    axs[2].imshow(hsv2rgb(img_eq_3),cmap='gray')
    axs[2].set_axis_off()

    axs[3].set_title("Cuantifiación")
    axs[3].imshow(quant4,cmap='gray')
    axs[3].set_axis_off()

    axs[4].set_title("Binarización y clausura")
    axs[4].imshow(quant4b,cmap='gray')
    axs[4].set_axis_off()

    plt.tight_layout()
    plt.show()
    

## Segmentación

In [42]:
# Funcion de segmentacion
def segmentacion(img):
    segmented = label(img, connectivity= None)
    props = regionprops(segmented)
    for prop in props:
        if prop.area > 10000 and prop.label != 0:
            region_label = prop.label
            region = (segmented == region_label)*1
            centroide = prop.centroid
    return region, centroide

if INF:
    index = 17#np.random.randint(0,len(lista_fotos))
    print('IMG_' + str(index+1) + '.png')
    quant4 = preproc(lista_fotos[index])
    reg = segmentacion(quant4)

    fig1, axs = plt.subplots(1,2,figsize=(9,6))
    fig1.suptitle('Etapa de segmentación')

    axs[0].set_title("Imagen preprocesada")
    axs[0].imshow(quant4,cmap='gray')
    axs[0].set_axis_off()

    axs[1].set_title("Imagen segmentada")
    axs[1].imshow(reg,cmap='gray')
    axs[1].set_axis_off()


    plt.tight_layout()
    plt.show()
    


## Detección de caminos

In [29]:
# Tipo de interseccion y esquinas
def intersecciones(region):
    r_border = region[:,300:320]
    l_border = region[:,0:20]
    u_border = region[0:40,:] # indica posible interseccion cruz o T
    ur_corner = region[0:20,300:320]
    ul_corner = region[0:20,0:20]
    r_path = np.sum(r_border) > 800 #existe camino der
    l_path = np.sum(l_border) > 800 #existe camino izq
    u_path = np.sum(u_border) > 500 #posible camino recto
    empty_corners = (np.sum(ur_corner) < 100) and (np.sum(ul_corner) < 100) #pocos o nulos pixeles en las esquinas
    #es decir la interseccion no queda en el borde para distinguir entre codo e interseccion

    #print(f"Pixeles en borde: \n derecho: {np.sum(r_border)} \n izquierdo: {np.sum(l_border)} \n superior: {np.sum(u_border)} \n esq der: {np.sum(ur_corner)} \n esq izq: {np.sum(ul_corner)}")
    #print(f"Pixeles en borde: \n derecho: {r_path} \n izquierdo: {l_path} \n superior: {u_path} \n esquinas vacias: {empty_corners}")


    #Intersecciones
    cruz = l_path and r_path and u_path
    Te = l_path and r_path and (not u_path)
    i_l = l_path and (not r_path) and u_path
    i_r = (not l_path) and r_path and u_path
    codo_l = l_path and (not r_path) and (not u_path)
    codo_r = (not l_path) and r_path and (not u_path)
    #print(f"Intersecciones: \n cruz: {cruz} \n T: {Te} \n izquierda: {i_l} \n derecha: {i_r} \n codo izq: {codo_l} \n codo der: {codo_r}")
    return (cruz, Te, i_l, i_r, codo_l, codo_r, empty_corners)

In [28]:
# Toma una instruccion y una interseccion y toma decision
def decision(inst, intersecciones):
    giro = False
    cruz = intersecciones[0]
    Te = intersecciones[1]
    i_l = intersecciones[2]
    i_r = intersecciones[3]
    codo_l = intersecciones[4]
    codo_r = intersecciones[5]
    empty_corners = intersecciones[6]
    if inst == "l" and empty_corners:
        if cruz or Te or i_l:
            print("Gire a la izq \n Girando \n")
            time.sleep(5)
            giro = True
        else:
            print("Continue \n")
    elif inst == "r" and empty_corners:
        if cruz or Te or i_r:
            print("Gire a la derecha \n Girando \n")
            time.sleep(5)
            giro = True
        else:
            print("Continue \n")
    elif inst == "c" and empty_corners:
        if cruz or Te or i_r:
            print("Gire a la derecha \n")
            print("Girando \n")
            #time.sleep(5)
            giro = True
        elif i_l or codo_r:
            print("Continue \n")
        else:
            print("Continue \n")
    else:
        print("Continue \n")
    return giro

In [ ]:
# Toma una instruccion y una interseccion y toma decision
def decision(inst, intersecciones):
    giro = False
    cruz = intersecciones[0]
    Te = intersecciones[1]
    i_l = intersecciones[2]
    i_r = intersecciones[3]
    codo_l = intersecciones[4]
    codo_r = intersecciones[5]
    empty_corners = intersecciones[6]
    if inst == "l" and empty_corners and (cruz or Te or i_l):
        print("Gire a la izq \n Girando \n")
        time.sleep(5)
        giro = True
    elif inst == "r" and empty_corners (cruz or Te or i_r):
        print("Gire a la derecha \n Girando \n")
        time.sleep(5)
        giro = True
    elif inst == "c" and empty_corners:
        if cruz or Te or i_r:
            print("Gire a la derecha \n")
            print("Girando \n")
            #time.sleep(5)
            giro = True
        elif i_l or codo_r:
            print("Continue \n")
        else:
            print("Continue \n")
    else:
        print("Continue \n")
    return giro

### Funcion de intersecciones y desiciones finales

In [50]:
# Tipo de interseccion y esquinas
def intersecciones2(region):
    r_border = region[:,300:320]
    l_border = region[:,0:20]
    u_border = region[0:40,:] # indica posible interseccion cruz o T
    ur_corner = region[0:20,300:320]
    ul_corner = region[0:20,0:20]
    r_path = np.sum(r_border) > 800 #existe camino der
    l_path = np.sum(l_border) > 800 #existe camino izq
    u_path = np.sum(u_border) > 500 #posible camino recto
    empty_corners = (np.sum(ur_corner) < 100) and (np.sum(ul_corner) < 100) #pocos o nulos pixeles en las esquinas
    #es decir la interseccion no queda en el borde para distinguir entre codo e interseccion
    #Intersecciones
    cruz = l_path and r_path and u_path
    Te = l_path and r_path and (not u_path)
    i_l = l_path and (not r_path) and u_path
    i_r = (not l_path) and r_path and u_path
    codo_l = l_path and (not r_path) and (not u_path)
    codo_r = (not l_path) and r_path and (not u_path)
    recto = (not l_path) and (not r_path) and u_path
    intersecciones = [cruz, Te, i_l, i_r, codo_l, codo_r, recto]
    interseccion = intersecciones.index(True)
    return interseccion, empty_corners

    
# Toma una instruccion y una interseccion y toma decision
def decision2(inst, interseccion, empty_corners):
    giro = False
    if empty_corners:
        #Cruz
        if interseccion == 0:
            if inst == "l":
                print("Gire a la izq \n Girando \n")
                time.sleep(5)
                giro = True
            elif inst == "r":
                print("Gire a la der \n Girando \n")
                time.sleep(5)
                giro = True
            elif inst == "c":
                print("Continue directo \n")
                time.sleep(5)
                giro = True
        #Te
        elif interseccion == 1:
            if inst == "l":
                print("Gire a la izq \n Girando \n")
                time.sleep(5)
                giro = True
            elif inst == "r":
                print("Gire a la der \n Girando \n")
                time.sleep(5)
                giro = True
            #Si se inidica continuar y solo se puede girar
            elif inst == "c":
                print("Intruccion no valida \n")
                print("Tome otra direccion \n")
                time.sleep(5)                
        #Interseccion izq
        elif interseccion == 2:
            if inst == "l":
                print("Gire a la izq \n Girando \n")
                time.sleep(5)
                giro = True
            elif inst == "r":
                print("Intruccion no valida \n")
                print("Continue directo \n")
                time.sleep(5)
            elif inst == "c":
                print("Continue directo \n")
                time.sleep(5)
                giro = True
        #Interseccion der
        elif interseccion == 3:
            if inst == "l":
                print("Intruccion no valida \n")
                print("Continue directo \n")
                time.sleep(5)            
            elif inst == "r":
                print("Gire a la der \n Girando \n")
                time.sleep(5)
                giro = True
            elif inst == "c":
                print("Continue directo \n")
                time.sleep(5)
                giro = True        
        #Codo izq/der
        elif interseccion == 4 or interseccion == 5:
            print("Codo detectado \n Continue \n")
            time.sleep(5)
        #Recto
        elif interseccion == 6:
            print("Continue \n")
    else:
        print("Continue \n")
    return giro

In [78]:
#Correccion de trayectoria
def correccion(centroide):
    x_c = centroide[1]
    #Se determina la distancia en x del centroide al centro de la imagen y 
    #si el desvio es muy alto indica que se debe hacer una correccion
    desviacion = abs(x_c - 160)
    if desviacion >= 20:
        print("pip, pip, pip")
        #playsound("pipipip.mp3")


## Pruebas parciales

In [30]:
folder = 'SECUENCIA'
folder_path = os.path.join(path, folder)
files = os.listdir(folder_path)
lista_fotos = []
for file in files:
    file_path = os.path.join(folder_path, file)
    foto = io.imread(file_path)
    lista_fotos.append(foto)

instrucciones = ["l","l","r","r"]
i = 0
while i < len(instrucciones):
    for index in range(len(lista_fotos)):
        print('IMG_' + str(index+1) + '.png ' + str(i))
        quant = preproc(lista_fotos[index])
        region = segmentacion(quant)
        #plt.imshow(region, cmap = "gray")
        #plt.show()
        intersecs = intersecciones(region)
        giro = decision (instrucciones[i],intersecs)
        if giro:
            i +=1
        if i == len(instrucciones):
            break
print("Se ejecutaron todas las instrucciones")



IMG_1.png 0
Continue 

IMG_2.png 0
Continue 

IMG_3.png 0
Continue 

IMG_4.png 0
Continue 

IMG_5.png 0
Continue 

IMG_6.png 0
Continue 

IMG_7.png 0
Gire a la izq 
 Girando 

IMG_8.png 1
Gire a la izq 
 Girando 

IMG_9.png 2
Continue 

IMG_10.png 2
Continue 

IMG_11.png 2
Gire a la derecha 
 Girando 

IMG_12.png 3
Continue 

IMG_13.png 3
Continue 

IMG_14.png 3
Gire a la derecha 
 Girando 

Se ejecutaron todas las instrucciones


In [54]:
folder = 'SECUENCIA'
folder_path = os.path.join(path, folder)
files = os.listdir(folder_path)
lista_fotos = []
for file in files:
    file_path = os.path.join(folder_path, file)
    foto = io.imread(file_path)
    lista_fotos.append(foto)

instrucciones = ["l","l","r","r"]
i = 0
while i < len(instrucciones):
    for index in range(len(lista_fotos)):
        print('IMG_' + str(index+1) + '.png ' + str(i))
        quant = preproc(lista_fotos[index])
        region, centroide = segmentacion(quant)
        #plt.imshow(region, cmap = "gray")
        #plt.show()
        intersec, esquinas = intersecciones2(region)
        if intersec == 6:
            print(centroide)
            correccion(centroide)
        giro = decision2(instrucciones[i], intersec, esquinas)
        if giro:
            i +=1
        if i == len(instrucciones):
            break
print("Se ejecutaron todas las instrucciones")

IMG_1.png 0
(196.35953196678474, 163.51469136519367)
Continue 

IMG_2.png 0
(184.33472611130557, 168.67231799089956)
Continue 

IMG_3.png 0
(187.40814489223112, 163.40896573637664)
Continue 

IMG_4.png 0
(200.5104965346369, 155.97077041550875)
Continue 

IMG_5.png 0
(198.44161140998372, 143.87779306893532)
Continue 

IMG_6.png 0
(196.35953196678474, 163.51469136519367)
Continue 

IMG_7.png 0
Gire a la izq 
 Girando 

IMG_8.png 1
Gire a la izq 
 Girando 

IMG_9.png 2
(193.39603755635522, 162.21474955536254)
Continue 

IMG_10.png 2
(187.40814489223112, 163.40896573637664)
Continue 

IMG_11.png 2
Gire a la der 
 Girando 

IMG_12.png 3
(195.74430442100078, 177.1509790238496)
Continue 

IMG_13.png 3
(195.74430442100078, 177.1509790238496)
Continue 

IMG_14.png 3
Gire a la der 
 Girando 

Se ejecutaron todas las instrucciones
